# Heterointerfaces (Interfaces between two different materials)

Taken from https://github.com/materialsproject/workshop/blob/master/workshop/lessons/03_heterointerfaces/Main%20Lesson.ipynb

This lesson focuses on a new feature in pymatgen, which is the construction of heterointerface structures. Heterointerfaces are interfaces between two materials that don't have to share chemistry or structure. They are particularly difficult to model due to the number of different ways in which they can be constructed. In some ways, crystalline features such as stacking faults and grain boundaries represent the simplest type of heterointerface.

To teach students how to build and manipulat heterointerfaces, we'll begin by focusing on building surfaces, move to understanding epitaxial matching, a critical algorithmic capability for heterointerface generation and finally finish by building some heterointerfaces ourselves.

## Lesson 1: Surfaces

One way to construct an interface is to take two surfaces and join them together. We'll begin by focusing on the tools in pymatgen that let us construct and manipulate surfaces. Since crystal structures in pymatgen are periodic all directions, surface structures are also technically in all dimensions. In order to build a free surface we introduce free space or "vacuum" into the cell so that there is a break in the periodicity and this "free surface" is now something we can compute properties of using DFT. We call this type of structure a Slab. Slabs are not easy to make, so most of the time we'll beging by using the SlabGenerator to construct slabs

Let's start by importing the LiFePO4 structure

In [1]:
from pymatgen.core.structure import Structure
LiFePO4 = Structure.from_file("LiFePO4.cif")


In [2]:
# !pip install crystaltoolkit-extension
import crystal_toolkit
LiFePO4

If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 4.746441 6.090226 10.44373
 angles : 90.0 90.0 90.0
 volume : 301.8958418773316
      A : 4.746441 0.0 2.906356888995881e-16
      B : 9.793830372249544e-16 6.090226 3.7291878884919943e-16
      C : 0.0 0.0 10.44373
    pbc : True True True
PeriodicSite: Li (0.0000, 0.0000, 5.2219) [0.0000, 0.0000, 0.5000]
PeriodicSite: Li (2.3732, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Li (2.3732, 3.0451, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Li (0.0000, 3.0451, 5.2219) [0.0000, 0.5000, 0.5000]
PeriodicSite: Fe (2.2557, 4.5677, 7.4988) [0.4752, 0.7500, 0.7180]
PeriodicSite: Fe (0.1175, 4.5677, 2.2770) [0.0248, 0.7500, 0.2180]
PeriodicSite: Fe (4.6290, 1.5226, 8.1668) [0.9752, 0.2500, 0.7820]

Let's add some oxidation states to LiFePO4, this will be important when we want to take surface polarity into consideration:

In [7]:
LiFePO4.add_oxidation_state_by_element({"Fe": 2, "Li": 1, "P": 5, "O": -2})

Let's now try to generate surfaces using the SlabGenerator class.

In [8]:
from pymatgen.core.surface import SlabGenerator

We can now initialize an instance of the SlabGeneratorclass, let's choose a Miller index of (0, 0, 1), a minimum slab height of 10 Å, and minimum 10 Å of vacuum.

In [9]:
slabgen = SlabGenerator(LiFePO4,
                        miller_index=(0,0,1),
                        min_slab_size=10,
                        min_vacuum_size=10,
                        center_slab=True)



We can now use the get_slabs() method of the SlabGenerator class to create a list of Slab objects. The slabs returned by get_slabs() in this case correspond to all the unique terminations along the normal to the Miller plane we are interested in. The different resulting slabs are characterized by different c shifts:

In [10]:
slabs = slabgen.get_slabs()

In [11]:
len(slabs)

5

In [12]:
print(slabs[0])

Slab Summary (Li4 Fe4 P4 O16)
Reduced Formula: LiFePO4
Miller index: (0, 0, 1)
Shift: 0.0215, Scale Factor: [[1 0 0]
 [0 1 0]
 [0 0 1]]
abc   :   4.746441   6.090226  20.887460
angles:  90.000000  90.000000  90.000000
Sites (28)
1 Li+     0.000000     0.000000     0.482143
2 Li+     0.500000     0.000000     0.732143
3 Li+     0.500000     0.500000     0.732143
4 Li+     0.000000     0.500000     0.482143
5 Fe2+     0.475247     0.750000     0.591155
6 Fe2+     0.024753     0.750000     0.341155
7 Fe2+     0.975247     0.250000     0.623131
8 Fe2+     0.524753     0.250000     0.373131
9 P5+     0.417816     0.250000     0.529520
10 P5+     0.917816     0.750000     0.684766
11 P5+     0.082184     0.250000     0.279520
12 P5+     0.582184     0.750000     0.434766
13 O2-     0.741866     0.250000     0.530504
14 O2-     0.241866     0.750000     0.683782
15 O2-     0.758134     0.250000     0.280504
16 O2-     0.258134     0.750000     0.433782
17 O2-     0.206946     0.250000     0.7

In [13]:
slabs[0]

If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 4.746441 6.090226 20.88746
 angles : 90.0 90.0 90.0
 volume : 603.7916837546632
      A : 4.746441 0.0 2.906356888995881e-16
      B : 9.793830372249544e-16 6.090226 3.7291878884919943e-16
      C : 0.0 0.0 20.88746
    pbc : True True True
PeriodicSite: Li+ (0.0, 0.0, 10.07) [0.0, 0.0, 0.4821]
PeriodicSite: Li+ (2.373, 0.0, 15.29) [0.5, 0.0, 0.7321]
PeriodicSite: Li+ (2.373, 3.045, 15.29) [0.5, 0.5, 0.7321]
PeriodicSite: Li+ (4.897e-16, 3.045, 10.07) [1.233e-32, 0.5, 0.4821]
PeriodicSite: Fe2+ (2.256, 4.568, 12.35) [0.4752, 0.75, 0.5912]
PeriodicSite: Fe2+ (0.1175, 4.568, 7.126) [0.02475, 0.75, 0.3412]
PeriodicSite: Fe2+ (4.629, 1.523, 13.02) [0.9752, 0.25, 0.6231]
PeriodicSite: Fe2+ (2.491, 1.523, 

Slabs also have a number of unique properties that are important when simulating them. Two very important properties are whether they are symmetric and polar. Polar surfaces can be more difficult to relax and compute because they naturally have a redistribution of charge. There are tricks that can be played in many DFT codes to fix this, but all cause other problems. Non-symmetric slabs make computing surface energies more difficult, as you can only compute the average surface energy of the two surfaces together. Let's run a loop over our surfaces and see which ones are polar and which ones are symmetric.

In [14]:
for n, slab in enumerate(slabs):
    print(n, "Polar:" ,slab.is_polar(), "Symmetric: ", slab.is_symmetric())

0 Polar: True Symmetric:  False
1 Polar: True Symmetric:  False
2 Polar: True Symmetric:  False
3 Polar: True Symmetric:  False
4 Polar: False Symmetric:  True


### Exercise 1:
Try getting the (111) slabs for Si. How many are there? 

In [15]:
# Solution
silicon = Structure.from_file("Si.cif")
slabgen = SlabGenerator(silicon, (1,1,1), 10, 10)
slabs = slabgen.get_slabs()
print("Number of slabs:", len(slabs))

Number of slabs: 2


In [17]:
# Now let's visualize the slabs to see what they look like
slabs[0]

If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 3.866974653264745 3.8669746532647444 25.357448563817616
 angles : 85.6269933583793 85.6269933583793 59.99999999999999
 volume : 327.1063427893186
      A : 3.855717061245752 0.0 0.29485405306764534
      B : 1.9165845041280052 3.3456326598893043 0.29485405306764534
      C : 0.0 0.0 25.357448563817616
    pbc : True True True
PeriodicSite: Si (1.533, 0.8887, 24.33) [0.2656, 0.2656, 0.9531]
PeriodicSite: Si (3.517, 2.039, 23.74) [0.6094, 0.6094, 0.9219]
PeriodicSite: Si (3.698, 2.143, 21.38) [0.6406, 0.6406, 0.8281]
PeriodicSite: Si (5.682, 3.293, 20.79) [0.9844, 0.9844, 0.7969]
PeriodicSite: Si (0.09019, 0.05228, 17.84) [0.01562, 0.01563, 0.7031]
PeriodicSite: Si (2.074, 1.202, 17.25) [0.3594, 0.3594

In [18]:
slabs[1]

If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 3.866974653264745 3.8669746532647444 25.357448563817616
 angles : 85.6269933583793 85.6269933583793 59.99999999999999
 volume : 327.1063427893186
      A : 3.855717061245752 0.0 0.29485405306764534
      B : 1.9165845041280052 3.3456326598893043 0.29485405306764534
      C : 0.0 0.0 25.357448563817616
    pbc : True True True
PeriodicSite: Si (1.533, 0.8887, 24.33) [0.2656, 0.2656, 0.9531]
PeriodicSite: Si (3.517, 2.039, 23.74) [0.6094, 0.6094, 0.9219]
PeriodicSite: Si (3.698, 2.143, 21.38) [0.6406, 0.6406, 0.8281]
PeriodicSite: Si (5.682, 3.293, 20.79) [0.9844, 0.9844, 0.7969]
PeriodicSite: Si (0.09019, 0.05228, 17.84) [0.01562, 0.01563, 0.7031]
PeriodicSite: Si (2.074, 1.202, 17.25) [0.3594, 0.3594

# Lesson 2: Epitaxial Matching

Now that we can enumerate surfaces to compute, we have a dilema. We can get several unique surfaces per miller index. There are easily tens of unique and important miller indicies per material. There are hundreds of thousands of materials to chose from when making a heterointerface. This already yields over 10^15 combinations to choose from, which is not something we can compute. How do we reduce this?

One option is to limit ourselves to coherent interfaces. Since coherent interfaces have to repeat along the interface, they need to maintain a condition called the epitaxial constraint. This means that lattice vectors for the surfaces that come together have to repeat by some integer multiple. This is a math problem we can solve and reduce our choices!

It turns out there is a practical use for this kind of math. Often, when we want to grow a really nice high quality film of a material, we'll use the epitaxial constraint to preferably orient and align the growing film. Obviously this isn't easy and only works for some systems, so we have to find out which ones. Pymatgen has a nice tool called the substrate analyzer to do this. Let's try using it:

In [19]:
from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer

In [20]:
# Now, we need two structures to check for a substrate match. One is the film that we want to grow and another is the potential substrate. 
# For this exercise, let's use the Silicon structure that is already in this folder:
silicon = Structure.from_file("Si.cif")

In [21]:
#Now, let's make a substrate analyzer object and calculate some matches
sub_analyzer = SubstrateAnalyzer()
sub_analyzer.calculate(film=LiFePO4,substrate=silicon)

<generator object SubstrateAnalyzer.calculate at 0x7f38023e7ac0>

In [ ]:
# The substrate analyzer uses a python generator which is an efficient way of getting 
# just one match if we want any match or a the full list of matches if we want all of them. 
# The computational time required will scale with the number of matches we want to evaluate, 
# so we can choose to be as computationally stringest as necessary.

# For the purpose of this example, let's get them all by turning the generator into a list



In [22]:
matches = list(sub_analyzer.calculate(film=LiFePO4,substrate=silicon))
len(matches)


6

In [23]:
# These matches have different matching area and strains, which we can print out

for match in matches:
    print(match.match_area,match.von_mises_strain)

289.06898385666 0.017056087472307125
289.06898385666 0.017056087472307337
289.06898385666 0.017056087472307014
289.06898385666 0.01705608747230727
241.91971097798265 0.008953220319809196
241.91971097798265 0.008953220319809045


In [24]:
# Let's look at the first two matches to see if we can figure out what is different. 
matches[0].as_dict()

{'@module': 'pymatgen.analysis.interfaces.substrate_analyzer',
 '@class': 'SubstrateMatch',
 '@version': None,
 'film_sl_vectors': array([[ 4.74644100e+00,  1.21804520e+01,  1.03647327e-15],
        [-1.89857640e+01,  1.21804520e+01, -4.16705178e-16]]),
 'substrate_sl_vectors': array([[-10.937456,   5.468728,  -5.468728],
        [-16.406184, -10.937456,  10.937456]]),
 'film_vectors': [array([4.74644100e+00, 0.00000000e+00, 2.90635689e-16]),
  array([9.79383037e-16, 6.09022600e+00, 3.72918789e-16])],
 'substrate_vectors': [array([5.46872800e+00, 0.00000000e+00, 3.34863012e-16]),
  array([ 8.79438536e-16,  5.46872800e+00, -5.46872800e+00])],
 'film_transformation': array([[ 1.,  2.],
        [ 0., 10.]]),
 'substrate_transformation': array([[1., 3.],
        [0., 7.]]),
 'film_miller': [0, 0, 1],
 'substrate_miller': [1, 1, 0],
 'strain': {'@module': 'pymatgen.analysis.elasticity.strain',
  '@class': 'Strain',
  'input_array': [[0.020074479545137084,
    -3.3477325344072135e-17,
    -0

In [25]:
# Now let's see what else can provide the substrate analyzer to understand how we can use it
SubstrateAnalyzer.calculate?

Signature:
SubstrateAnalyzer.calculate(
    self,
    film,
    substrate,
    elasticity_tensor=None,
    film_millers=None,
    substrate_millers=None,
    ground_state_energy=0,
    lowest=False,
)
Docstring:
Finds all topological matches for the substrate and calculates elastic
strain energy and total energy for the film if elasticity tensor and
ground state energy are provided:

Args:
    film(Structure): conventional standard structure for the film
    substrate(Structure): conventional standard structure for the
        substrate
    elasticity_tensor(ElasticTensor): elasticity tensor for the film
        in the IEEE orientation
    film_millers(array): film facets to consider in search as defined by
        miller indices
    substrate_millers(array): substrate facets to consider in search as
        defined by miller indices
    ground_state_energy(float): ground state energy for the film
    lowest(bool): only consider lowest matching area for each surface
File:      ~/jupyte

In [26]:
# We can see from the help text that SubstrateAnayzer inherits from ZSLGenerator,
# maybe we can understand from that how to tweak the parameters:

from pymatgen.analysis.interfaces.zsl import ZSLGenerator

ZSLGenerator?


Init signature:
ZSLGenerator(
    max_area_ratio_tol=0.09,
    max_area=400,
    max_length_tol=0.03,
    max_angle_tol=0.01,
    bidirectional=False,
)
Docstring:     
This class generate matching interface super lattices based on the methodology
of lattice vector matching for heterostructural interfaces proposed by
Zur and McGill:
Journal of Applied Physics 55 (1984), 378 ; doi: 10.1063/1.333084
The process of generating all possible matching super lattices is:
1.) Reduce the surface lattice vectors and calculate area for the surfaces
2.) Generate all super lattice transformations within a maximum allowed area
    limit that give nearly equal area super-lattices for the two
    surfaces - generate_sl_transformation_sets
3.) For each superlattice set:
    1.) Reduce super lattice vectors
    2.) Check length and angle between film and substrate super lattice
        vectors to determine if the super lattices are the nearly same
        and therefore coincident - get_equiv_transformati

## Exercise 2:

Try using the substrate analyzer to look at a specific substrate and film orientation:

(1,0,0) LiFePO4 on (1,1,1) Si

Note the default tolerances aren't big enough to get any matches here. You'll need to increase them as follows:

    max_area: 800

Try increasing the max_area, max_length_tol and max_angle_tol. How does these seem to affect the results?

In [27]:
# Solution:
sub_analyzer = SubstrateAnalyzer(max_area_ratio_tol=0.09,
                                 max_area=800,
                                 max_length_tol=0.03,
                                 max_angle_tol=0.01,
                                 bidirectional=False
                                )

matches = list(sub_analyzer.calculate(film=LiFePO4,
                                      substrate=silicon,
                                      film_millers=[(1,0,0)],
                                      substrate_millers=[(1,1,1)]))
len(matches)

2

# Building and Manipulating heterointerfaces


Pymatgen has some capabillitis to build coherent heterointerfaces. This enables us to directly simulate the properties of heterointerfaces. Currently this capabilty is limited to coherent heterointerfaces using the epitaxial constraint to find matching lattices. While this does reduce the number of choices by focusing only on systems that can build a matching repeat within specific tolerances. It's often possible to find hundreds of matches.

Typically lower match area is a good thing as it means that the repeats are closer together and more likely to force the interface to stay coherent. If the repeat distance is large, there are no short to medium range interactions that the two surfaces feel to force coherency. The other issue is epitaxial strain. Typically when synthesizing a thin film, this epitaxial strain has a natural limitation because it costs energy to maintain. If the elastic tensor of the film is already known, it's possible to compute this "elastic energy", whcih can easily be on the order of a 10's of meV, which is inline with polymorph selection. Thus at some point the epitaxial strain is too costly, so the film changes phase or loses coherency.

With these guidelines in place, let's build a heterointerface

In [28]:
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder

In [29]:
# Let's build an interface builder for our LiFePO4 and silicon, using the information we learned from the substrate analyzer. 
# That means we need to use the ZSL generator from the exercise to get some matches.

zsl = ZSLGenerator(max_area=800)

In [30]:
cib = CoherentInterfaceBuilder(film_structure=LiFePO4,
                               substrate_structure=silicon,
                               film_miller=(1,1,1),
                               substrate_miller=(1,0,0),
                               zslgen=zsl)

In [31]:
# The CoherentInterfaceBuilder first finds all the unique terminations that can be exposed. 
# Let's look at those first:

cib.terminations

[('Li2Fe3O5_P1_10', 'Si_P4/mmm_1'),
 ('Li2Fe4O5_P1_11', 'Si_P4/mmm_1'),
 ('LiFe2O3_P-1_12', 'Si_P4/mmm_1'),
 ('O2_P-1_1', 'Si_P4/mmm_1'),
 ('P_P-1_1', 'Si_P4/mmm_1'),
 ('PO4_P1_5', 'Si_P4/mmm_1')]

In [32]:
cib.get_interfaces?

Signature:
cib.get_interfaces(
    termination: 'tuple[str, str]',
    gap: 'float' = 2.0,
    vacuum_over_film: 'float' = 20.0,
    film_thickness: 'float | int' = 1,
    substrate_thickness: 'float | int' = 1,
    in_layers: 'bool' = True,
) -> 'Iterator[Interface]'
Docstring:
Generates interface structures given the film and substrate structure
as well as the desired terminations

Args:
    termination (tuple[str, str]): termination from self.termination list
    gap (float, optional): gap between film and substrate. Defaults to 2.0.
    vacuum_over_film (float, optional): vacuum over the top of the film. Defaults to 20.0.
    film_thickness (float | int, optional): the film thickness. Defaults to 1.
    substrate_thickness (float | int, optional): substrate thickness. Defaults to 1.
    in_layers (bool, optional): set the thickness in layer units. Defaults to True.

Yields:
    Iterator[Interface]: interfaces from slabs
File:      ~/jupyter/notebookenv/lib/python3.8/site-packages/p

In [33]:
interfaces=list(cib.get_interfaces(termination= ('O2_P-1_1', 'Si_P4/mmm_1')))
len(interfaces)


30

In [34]:
interface = interfaces[0]
# This just fixes the atomic coordinates so that structure looks nice in the 3D Viewer
interface.translate_sites(range(len(interface)),[0,0,0]) 
interface

If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 7.733949306529491 54.137645145706436 29.285478443623763
 angles : 90.0 90.0 90.0
 volume : 12261.76548800825
      A : 7.733949306529491 0.0 4.735678131504616e-16
      B : 8.705997335896692e-15 54.137645145706436 3.3149746920532316e-15
      C : 0.0 0.0 29.285478443623763
    pbc : True True True
PeriodicSite: Li+ (4.132, 50.88, 17.98) [0.5343, 0.9399, 0.6138]
PeriodicSite: Li+ (5.679, 18.4, 17.98) [0.7343, 0.3399, 0.6138]
PeriodicSite: Li+ (1.039, 7.571, 17.98) [0.1343, 0.1399, 0.6138]
PeriodicSite: Li+ (7.226, 40.05, 17.98) [0.9343, 0.7399, 0.6138]
PeriodicSite: Li+ (2.586, 29.23, 17.98) [0.3343, 0.5399, 0.6138]
PeriodicSite: Li+ (2.586, 2.158, 17.98) [0.3343, 0.03985, 0.6138]
PeriodicSite: Li+ (4

In [35]:
interface.gap

2.0

In [37]:
interface.gap=5.0
interface

If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 7.733949306529491 54.137645145706436 32.28547844362376
 angles : 90.0 90.0 90.0
 volume : 13517.858897403576
      A : 7.733949306529491 0.0 4.735678131504616e-16
      B : 8.705997335896692e-15 54.137645145706436 3.3149746920532316e-15
      C : 0.0 0.0 32.28547844362376
    pbc : True True True
PeriodicSite: Li+ (4.132, 50.88, 20.98) [0.5343, 0.9399, 0.6497]
PeriodicSite: Li+ (5.679, 18.4, 20.98) [0.7343, 0.3399, 0.6497]
PeriodicSite: Li+ (1.039, 7.571, 20.98) [0.1343, 0.1399, 0.6497]
PeriodicSite: Li+ (7.226, 40.05, 20.98) [0.9343, 0.7399, 0.6497]
PeriodicSite: Li+ (2.586, 29.23, 20.98) [0.3343, 0.5399, 0.6497]
PeriodicSite: Li+ (2.586, 2.158, 20.98) [0.3343, 0.03985, 0.6497]
PeriodicSite: Li+ (4.

In [38]:
# As we can see, that opened up a lot of space in between the two slabs. 
# We can also modify inplane_offset to control how the two slabs are oriented along each other 
# in the directions perpendicular to the surfaces. 
# Notice that the size of the box didn't change. What if we change the vacuum.

interface.vacuum_over_film 

20.0

In [39]:
interface.vacuum_over_film = 100
interface

If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 7.733949306529491 54.137645145706436 112.28547844362376
 angles : 90.0 90.0 90.0
 volume : 47013.68314794565
      A : 7.733949306529491 0.0 4.735678131504616e-16
      B : 8.705997335896692e-15 54.137645145706436 3.3149746920532316e-15
      C : 0.0 0.0 112.28547844362376
    pbc : True True True
PeriodicSite: Li+ (4.132, 50.88, 20.98) [0.5343, 0.9399, 0.1868]
PeriodicSite: Li+ (5.679, 18.4, 20.98) [0.7343, 0.3399, 0.1868]
PeriodicSite: Li+ (1.039, 7.571, 20.98) [0.1343, 0.1399, 0.1868]
PeriodicSite: Li+ (7.226, 40.05, 20.98) [0.9343, 0.7399, 0.1868]
PeriodicSite: Li+ (2.586, 29.23, 20.98) [0.3343, 0.5399, 0.1868]
PeriodicSite: Li+ (2.586, 2.158, 20.98) [0.3343, 0.03985, 0.1868]
PeriodicSite: Li+ (4